## 匯入套件和資料

In [1]:
import os
import jieba
import numpy as np
import csv

In [2]:
def csv_create(file_name,model,co_id,list_name):
    with open(file_name,model,newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['公司名稱','95,96年','96,97年','97,98年','98,99年','99,100年','100,101年','101,102年','102,103年','103,104年','104,105年','105,106年','106,107年','107,108年','108,109年'])
        writer.writerow([co_id]+list_name)

In [3]:
all_file = []
txt_root = 'C:\\Users\\DC\\中文詞向量\\1909文字'

for root, dirs, files in os.walk(txt_root): 
    for file in files: 
        print(root + '\\' + file)
        all_file.append(root + '\\' + file)

C:\Users\DC\中文詞向量\1909文字\2006.txt
C:\Users\DC\中文詞向量\1909文字\2007.txt
C:\Users\DC\中文詞向量\1909文字\2008.txt
C:\Users\DC\中文詞向量\1909文字\2009.txt
C:\Users\DC\中文詞向量\1909文字\2010.txt
C:\Users\DC\中文詞向量\1909文字\2011.txt
C:\Users\DC\中文詞向量\1909文字\2012.txt
C:\Users\DC\中文詞向量\1909文字\2013.txt
C:\Users\DC\中文詞向量\1909文字\2014.txt
C:\Users\DC\中文詞向量\1909文字\2015.txt
C:\Users\DC\中文詞向量\1909文字\2016.txt
C:\Users\DC\中文詞向量\1909文字\2017.txt
C:\Users\DC\中文詞向量\1909文字\2018.txt
C:\Users\DC\中文詞向量\1909文字\2019.txt
C:\Users\DC\中文詞向量\1909文字\2020.txt


In [4]:
all_txt = []

for i in range(0,len(all_file)):
    txt = open(all_file[i], 'r', encoding='utf-8').read()
    txt = txt.replace('■','').replace('、','').replace('：','').replace('\n','').replace(' ','').replace('●','').replace('，','').replace('│','').replace('。','').replace(',','').replace(';','').replace('(','').replace(')','').replace('；','')
    a =jieba.lcut(txt)
    all_txt.append(a)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\DC\AppData\Local\Temp\jieba.cache
Loading model cost 0.425 seconds.
Prefix dict has been built successfully.


## 載入模型後將文字轉為向量！

In [5]:
from gensim.models import word2vec
model = word2vec.Word2Vec.load('wiki.model') #載入模型

In [6]:
padding = np.zeros((model.vector_size,), dtype=np.float32) #0向量
all_vec_add = []
vec_add = 0

for j in range(0,len(all_txt)):
    for i in all_txt[j]:
        try:
            vec_add += model.wv[i]
        except:
            model.wv[i] = padding
            vec_add += model.wv[i]
    all_vec_add.append(vec_add)
    vec_add = 0
    
print(all_vec_add[1])

[ 1.00111389e+02 -6.59460373e+01  1.56108337e+02 -3.51718079e+02
 -5.57278564e+02 -2.06947525e+02  7.17156494e+02  4.41984650e+02
 -3.39753540e+02 -4.06402206e+01 -1.80314505e+00 -4.13722351e+02
 -1.30621201e+02 -1.47997345e+02 -2.29910004e+02  1.47077393e+02
  5.06093323e+02  4.12250481e+01 -1.49780823e+02 -3.64934082e+01
 -5.49533081e+01  4.45613365e+01  4.18395508e+02  8.24350281e+01
 -2.65339203e+02 -1.86800949e+02  4.88425598e+01 -1.54743973e+02
  1.63938217e+02  4.17414307e+02  2.89785828e+02 -1.37808853e+02
  2.12703125e+02 -2.79436279e+02  1.94360771e+01  2.04358734e+02
 -3.46113930e+01  1.33338913e+02  2.03046188e+02 -1.64690033e+02
 -5.42668945e+02 -2.04688690e+02 -2.27373917e+02 -5.39764404e+00
  3.45741638e+02  4.84126740e+02  6.04484367e+01  3.27161072e+02
  8.37344055e+01  3.28993683e+01  1.27514336e+02  2.51649887e+02
  7.99854736e+01 -1.77639374e+02  1.78975327e+02  1.95451935e+02
  7.96420227e+02  3.95889587e+02 -4.78998169e+02 -1.91233749e+02
 -4.64798553e+02  1.35393

## 歐式計算距離寫入csv

In [7]:
#歐式
Euc = []

for i in range(0,len(all_vec_add)-1):
    x = all_vec_add[i]
    y = all_vec_add[i+1]
    dis = np.linalg.norm( all_vec_add[i] - all_vec_add[i+1])
    Euc.append(dis)
    print(dis)

618.22797
764.49585
1176.2107
874.5292
699.4596
648.46106
2304.8887
8787.078
1853.0522
2175.429
864.57776
1452.7853
1386.5842
1349.2218


In [45]:
csv_file = 'euc_per_year.csv'
write_mode = 'w'
co_id = '1909'

csv_create(csv_file,write_mode,co_id,Euc)

## 餘弦計算距離寫入csv

In [8]:
#餘弦
Man = []

for i in range(0,len(all_vec_add)-1):
    x = all_vec_add[i]
    y = all_vec_add[i+1]
    
    #小數方式
    #np.set_printoptions(precision=10,suppress=True) #位數
    #dis =  np.array([1 - np.dot(x,y)/(np.linalg.norm(x)*np.linalg.norm(y))])
    
    #科學記號方式
    dis =  1 - np.dot(x,y)/(np.linalg.norm(x)*np.linalg.norm(y))
    print(dis)
    Man.append(dis)

0.007903516292572021
0.012111008167266846
0.02953892946243286
0.020669639110565186
0.015810012817382812
0.013813555240631104
0.025684595108032227
0.01723700761795044
0.00725102424621582
0.004765689373016357
0.001683056354522705
0.005492687225341797
0.0034882426261901855
0.00240325927734375


In [32]:
csv_file = 'man_per_year.csv'
write_mode = 'w'
co_id = '1909'

csv_create(csv_file,write_mode,co_id,Man)

### 其他

In [75]:
similarity = model.wv.similarity('股東',all_txt[0][2]) #兩點距離
similarity

0.9796264

In [9]:
for item in model.wv.most_similar('營收'):
    print(item)

('銷售額', 0.7324838638305664)
('營業額', 0.7170495390892029)
('淨利潤', 0.7029321193695068)
('盈利', 0.6880814433097839)
('市占率', 0.6595735549926758)
('業績', 0.6550719738006592)
('市值', 0.6541926860809326)
('市佔率', 0.6344636082649231)
('產值', 0.6336631774902344)
('虧損', 0.6332756876945496)


C:\Users\DC\anaconda3\lib\site-packages\gensim\models\keyedvectors.py:783: RuntimeWarning: invalid value encountered in true_divide
  dists = dot(self.vectors[clip_start:clip_end], mean) / self.norms[clip_start:clip_end]


In [ ]:
#https://www.itread01.com/content/1548060671.html